In [ ]:
import sqlite3
import tkinter.messagebox
from tkinter import *

# Kết nối cơ sở dữ liệu SQLite
conn = sqlite3.connect('inventory.db')
c = conn.cursor()

# Tạo bảng hàng tồn kho nếu chưa có
c.execute('''CREATE TABLE IF NOT EXISTS inventory
             (id INTEGER PRIMARY KEY, name TEXT, price REAL, quantity INTEGER)''')
conn.commit()

# Hàm thêm hàng vào kho
def add_item():
    name = name_entry.get()
    price = float(price_entry.get())
    quantity = int(quantity_entry.get())
    c.execute("INSERT INTO inventory (name, price, quantity) VALUES (?, ?, ?)",
              (name, price, quantity))
    conn.commit()
    tkinter.messagebox.showinfo("Thông báo", "Đã thêm hàng vào kho.")

# Hàm xuất hàng khỏi kho
def remove_item():
    name = name_entry.get()
    quantity = int(quantity_entry.get())
    c.execute("SELECT quantity FROM inventory WHERE name=?", (name,))
    current_quantity = c.fetchone()[0]
    if current_quantity < quantity:
        tkinter.messagebox.showwarning("Cảnh báo", "Số lượng hàng trong kho không đủ.")
    else:
        new_quantity = current_quantity - quantity
        c.execute("UPDATE inventory SET quantity=? WHERE name=?", (new_quantity, name))
        conn.commit()
        tkinter.messagebox.showinfo("Thông báo", "Đã xuất hàng khỏi kho.")

# Hàm tìm kiếm hàng trong kho
def search_item():
    name = name_entry.get()
    c.execute("SELECT * FROM inventory WHERE name=?", (name,))
    result = c.fetchone()
    if result:
        price_entry.delete(0, END)
        price_entry.insert(END, str(result[2]))
        quantity_entry.delete(0, END)
        quantity_entry.insert(END, str(result[3]))
    else:
        tkinter.messagebox.showwarning("Cảnh báo", "Không tìm thấy hàng trong kho.")

# Hàm tạo hóa đơn bán hàng
def create_invoice():
    name = name_entry.get()
    quantity = int(quantity_entry.get())
    c.execute("SELECT * FROM inventory WHERE name=?", (name,))
    result = c.fetchone()
    if result:
        if result[3] < quantity:
            tkinter.messagebox.showwarning("Cảnh báo", "Số lượng hàng trong kho không đủ.")
        else:
            new_quantity = result[3] - quantity
            c.execute("UPDATE inventory SET quantity=? WHERE name=?", (new_quantity, name))
            conn.commit()
            total_price = result[2] * quantity
            tkinter.messagebox.showinfo("Thông báo", f"Đã bán {quantity} {name} với giá {total_price} đồng.")
    else:
        tkinter.messagebox.showwarning("Cảnh báo", "Không tìm thấy hàng trong kho.")

# Tạo giao diện
root = Tk()
root.title("Phần mềm Quản lý hàng tồn kho")

# Tạo các widget
name_label = Label(root, text="Tên hàng")
name_entry = Entry(root)

price_label = Label(root, text="Giá")
price_entry=Entry(root)

quantity_label = Label(root, text="Số lượng")
quantity_entry = Entry(root)

add_button = Button(root, text="Thêm hàng", command=add_item)
remove_button = Button(root, text="Xuất hàng", command=remove_item)
search_button = Button(root, text="Tìm kiếm", command=search_item)
create_invoice_button = Button(root, text="Tạo hóa đơn", command=create_invoice)

name_label.grid(row=0, column=0)
name_entry.grid(row=0, column=1)

price_label.grid(row=1, column=0)
price_entry.grid(row=1, column=1)

quantity_label.grid(row=2, column=0)
quantity_entry.grid(row=2, column=1)
add_button.grid(row=3, column=0)
remove_button.grid(row=3, column=1)
search_button.grid(row=4, column=0)
create_invoice_button.grid(row=4, column=1)

# Khởi động giao diện
root.mainloop()